In [4]:
import numpy as np
import pandas as pd
import nltk
from underthesea import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [5]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score

In [6]:
df = pd.read_csv('ViLegalDoc.csv')
df

,content,label
0,Phạm vi điều chỉnh: Luật này quy định về việc ...,LUẬT DOANH NGHIỆP
1,"Đối tượng áp dụng: Doanh nghiệp. Cơ quan, tổ c...",LUẬT DOANH NGHIỆP
2,Áp dụng Luật Doanh nghiệp và luật khác. Trường...,LUẬT DOANH NGHIỆP
3,Bảo đảm của Nhà nước đối với doanh nghiệp và c...,LUẬT DOANH NGHIỆP
4,"Tổ chức chính trị, tổ chức chính trị - xã hội ...",LUẬT DOANH NGHIỆP
...,...,...
1227,Điều 204. Trường hợp đình công bất hợp pháp\n\...,Luật lao động
1228,Điều 205. Thông báo quyết định đóng cửa tạm th...,Luật lao động
1229,Điều 206. Trường hợp cấm đóng cửa tạm thời nơi...,Luật lao động
1230,Điều 207. Tiền lương và các quyền lợi hợp pháp...,Luật lao động


# DATA PROCESSING

In [7]:
# val: 22 samples per class
# test: 27 samples per class
# train: the rest

def train_val_test_split(df, class_name):
    temp_df = df[df['label'] == class_name]

    test_df = temp_df.sample(n=27, random_state=42)
    temp_df = temp_df[~temp_df.index.isin(test_df.index)]

    val_df = temp_df.sample(n=22, random_state=42)
    train_df = temp_df[~temp_df.index.isin(val_df.index)]
    return train_df, val_df, test_df

In [8]:
df['label'].unique()

array(['LUẬT DOANH NGHIỆP', 'LUẬT VIÊN CHỨC', 'LUẬT PHÒNG, CHỐNG MA TÚY',
       'LUẬT BẢO Vệ MÔI TRƯỜNG', 'LUẬT ĐẤU THẦU', 'LUẬT GIÁO DỤC',
       'LUẬT ĐẤT ĐAI', 'Luật xử lý vi phạm hành chính', 'Luật lao động'],
      dtype=object)

In [9]:
df_train_1, df_val_1, df_test_1 = train_val_test_split(df=df, class_name='LUẬT DOANH NGHIỆP')
df_train_2, df_val_2, df_test_2 = train_val_test_split(df=df, class_name='LUẬT VIÊN CHỨC')
df_train_3, df_val_3, df_test_3 = train_val_test_split(df=df, class_name='LUẬT PHÒNG, CHỐNG MA TÚY')
df_train_4, df_val_4, df_test_4 = train_val_test_split(df=df, class_name='LUẬT BẢO Vệ MÔI TRƯỜNG')
df_train_5, df_val_5, df_test_5 = train_val_test_split(df=df, class_name='LUẬT ĐẤU THẦU')
df_train_6, df_val_6, df_test_6 = train_val_test_split(df=df, class_name='LUẬT GIÁO DỤC')
df_train_7, df_val_7, df_test_7 = train_val_test_split(df=df, class_name='LUẬT ĐẤT ĐAI')
df_train_8, df_val_8, df_test_8 = train_val_test_split(df=df, class_name='Luật xử lý vi phạm hành chính')
df_train_9, df_val_9, df_test_9 = train_val_test_split(df=df, class_name='Luật lao động')

In [10]:
df_train = pd.concat(objs=[df_train_1, df_train_2, df_train_3, df_train_4, df_train_5, df_train_6, df_train_7, df_train_8, df_train_9])
df_val = pd.concat(objs=[df_val_1, df_val_2, df_val_3, df_val_4, df_val_5, df_val_6, df_val_7, df_val_8, df_val_9])
df_test = pd.concat(objs=[df_test_1, df_test_2, df_test_3, df_test_4, df_test_5, df_test_6, df_test_7, df_test_8, df_test_9])

len(df_train), len(df_val), len(df_test)

(791, 198, 243)

In [11]:
stopword_file = "vietnamese-stopwords.txt"
with open(stopword_file, "r", encoding="utf-8") as f:
    stopword_content = f.read()

stop_words = stopword_content.splitlines()

In [12]:
# lower-casing - chuyen ve chu thuong
# tokenizing
# stopword removing
# numeric removing

def text_processing(content):
    cleaned_content = content.lower()
    cleaned_content = re.sub('[^a-zA-Zà-ỹĂÂĐÊÔƠƯơư]', ' ', cleaned_content)
    cleaned_content = word_tokenize(cleaned_content)
    cleaned_content = [word for word in cleaned_content if word not in stop_words]
    cleaned_content = ' '.join(cleaned_content)
    cleaned_content = re.sub(' +', ' ', cleaned_content)

    return  cleaned_content

In [13]:
df_train['cleaned_text'] = df_train['content'].apply(text_processing)
df_train

,content,label,cleaned_text
2,Áp dụng Luật Doanh nghiệp và luật khác. Trường...,LUẬT DOANH NGHIỆP,áp dụng luật doanh nghiệp luật trường hợp luật...
3,Bảo đảm của Nhà nước đối với doanh nghiệp và c...,LUẬT DOANH NGHIỆP,bảo đảm nhà nước đối với doanh nghiệp chủ sở h...
5,Quyền của doanh nghiệp. Tự do kinh doanh ngành...,LUẬT DOANH NGHIỆP,quyền doanh nghiệp tự do kinh doanh ngành nghề...
8,Người đại diện theo pháp luật của doanh nghiệp...,LUẬT DOANH NGHIỆP,đại diện pháp luật doanh nghiệp đại diện pháp ...
11,Điều 15. Trách nhiệm của người đại diện theo ủ...,LUẬT DOANH NGHIỆP,trách nhiệm đại diện ủy quyền chủ sở hữu thành...
...,...,...,...
1227,Điều 204. Trường hợp đình công bất hợp pháp\n\...,Luật lao động,trường hợp đình công bất hợp pháp trường hợp đ...
1228,Điều 205. Thông báo quyết định đóng cửa tạm th...,Luật lao động,thông báo quyết định đóng cửa tạm thời làm việ...
1229,Điều 206. Trường hợp cấm đóng cửa tạm thời nơi...,Luật lao động,trường hợp cấm đóng cửa tạm thời làm việc thời...
1230,Điều 207. Tiền lương và các quyền lợi hợp pháp...,Luật lao động,tiền lương quyền lợi hợp pháp lao động thời gi...


In [14]:
df_test['cleaned_text'] = df_test['content'].apply(text_processing)
df_test

,content,label,cleaned_text
62,"Điều 66. Tiền lương, thù lao, thưởng và lợi íc...",LUẬT DOANH NGHIỆP,tiền lương thù lao thưởng lợi ích chủ tịch hội...
40,"Điều 44. Chi nhánh, văn phòng đại diện và địa ...",LUẬT DOANH NGHIỆP,chi nhánh văn phòng đại diện địa điểm kinh doa...
93,Điều 97. Trách nhiệm của Chủ tịch và thành viê...,LUẬT DOANH NGHIỆP,trách nhiệm chủ tịch thành viên hội đồng thành...
18,Điều 22. Hồ sơ đăng ký công ty cổ phần. 1. Giấ...,LUẬT DOANH NGHIỆP,hồ sơ đăng ký công ty cổ phần giấy đề nghị đăn...
81,Điều 85. Cơ cấu tổ chức quản lý công ty trách ...,LUẬT DOANH NGHIỆP,cơ cấu tổ chức quản lý công ty trách nhiệm hữu...
...,...,...,...
1180,"Điều 157. Cấp, cấp lại, gia hạn, thu hồi giấy ...",Luật lao động,gia hạn thu hồi giấy phép lao động giấy xác nh...
1125,Điều 102. Khấu trừ tiền lương\n\n1. Người sử d...,Luật lao động,khấu trừ tiền lương sử dụng lao động khấu trừ ...
1097,Điều 66. Nguyên tắc thương lượng tập thể\n\nTh...,Luật lao động,nguyên tắc thương lượng tập thể thương lượng t...
1092,"Điều 61. Học nghề, tập nghề để làm việc cho ng...",Luật lao động,học nghề tập nghề làm việc sử dụng lao động họ...


In [15]:
df_val['cleaned_text'] = df_val['content'].apply(text_processing)
df_val

,content,label,cleaned_text
32,Điều 36. Định giá tài sản góp vốn. 1. Tài sản ...,LUẬT DOANH NGHIỆP,định giá tài sản góp vốn tài sản góp vốn đồng ...
1,"Đối tượng áp dụng: Doanh nghiệp. Cơ quan, tổ c...",LUẬT DOANH NGHIỆP,đối tượng áp dụng doanh nghiệp cơ quan tổ chức...
69,Điều 73. Công bố thông tin: Công ty trách nhiệ...,LUẬT DOANH NGHIỆP,công bố thông tin công ty trách nhiệm hữu hạn ...
6,Nghĩa vụ của doanh nghiệp. Doanh nghiệp xã hội...,LUẬT DOANH NGHIỆP,nghĩa vụ doanh nghiệp doanh nghiệp xã hội đáp ...
77,Điều 81. Chủ tịch công ty. 1. Chủ tịch công ty...,LUẬT DOANH NGHIỆP,chủ tịch công ty chủ tịch công ty chủ sở hữu c...
...,...,...,...
1094,Điều 63. Tổ chức đối thoại tại nơi làm việc\n\...,Luật lao động,tổ chức đối thoại làm việc đối thoại làm việc ...
1058,Điều 27. Kết thúc thời gian thử việc\n\n1. Khi...,Luật lao động,kết thúc thời gian thử kết thúc thời gian thử ...
1210,Điều 187. Thẩm quyền giải quyết tranh chấp lao...,Luật lao động,thẩm quyền giải quyết tranh chấp lao động cá n...
1140,Điều 117. Kỷ luật lao động\n\nKỷ luật lao động...,Luật lao động,kỷ luật lao động kỷ luật lao động quy định tuâ...


In [16]:
df_train.drop(columns=['content'], inplace=True)
df_train = df_train[['cleaned_text', 'label']]
df_train

,cleaned_text,label
2,áp dụng luật doanh nghiệp luật trường hợp luật...,LUẬT DOANH NGHIỆP
3,bảo đảm nhà nước đối với doanh nghiệp chủ sở h...,LUẬT DOANH NGHIỆP
5,quyền doanh nghiệp tự do kinh doanh ngành nghề...,LUẬT DOANH NGHIỆP
8,đại diện pháp luật doanh nghiệp đại diện pháp ...,LUẬT DOANH NGHIỆP
11,trách nhiệm đại diện ủy quyền chủ sở hữu thành...,LUẬT DOANH NGHIỆP
...,...,...
1227,trường hợp đình công bất hợp pháp trường hợp đ...,Luật lao động
1228,thông báo quyết định đóng cửa tạm thời làm việ...,Luật lao động
1229,trường hợp cấm đóng cửa tạm thời làm việc thời...,Luật lao động
1230,tiền lương quyền lợi hợp pháp lao động thời gi...,Luật lao động


In [17]:
df_test.drop(columns=['content'], inplace=True)
df_test = df_test[['cleaned_text', 'label']]
df_test

,cleaned_text,label
62,tiền lương thù lao thưởng lợi ích chủ tịch hội...,LUẬT DOANH NGHIỆP
40,chi nhánh văn phòng đại diện địa điểm kinh doa...,LUẬT DOANH NGHIỆP
93,trách nhiệm chủ tịch thành viên hội đồng thành...,LUẬT DOANH NGHIỆP
18,hồ sơ đăng ký công ty cổ phần giấy đề nghị đăn...,LUẬT DOANH NGHIỆP
81,cơ cấu tổ chức quản lý công ty trách nhiệm hữu...,LUẬT DOANH NGHIỆP
...,...,...
1180,gia hạn thu hồi giấy phép lao động giấy xác nh...,Luật lao động
1125,khấu trừ tiền lương sử dụng lao động khấu trừ ...,Luật lao động
1097,nguyên tắc thương lượng tập thể thương lượng t...,Luật lao động
1092,học nghề tập nghề làm việc sử dụng lao động họ...,Luật lao động


In [18]:
df_val.drop(columns=['content'], inplace=True)
df_val = df_val[['cleaned_text', 'label']]
df_val

,cleaned_text,label
32,định giá tài sản góp vốn tài sản góp vốn đồng ...,LUẬT DOANH NGHIỆP
1,đối tượng áp dụng doanh nghiệp cơ quan tổ chức...,LUẬT DOANH NGHIỆP
69,công bố thông tin công ty trách nhiệm hữu hạn ...,LUẬT DOANH NGHIỆP
6,nghĩa vụ doanh nghiệp doanh nghiệp xã hội đáp ...,LUẬT DOANH NGHIỆP
77,chủ tịch công ty chủ tịch công ty chủ sở hữu c...,LUẬT DOANH NGHIỆP
...,...,...
1094,tổ chức đối thoại làm việc đối thoại làm việc ...,Luật lao động
1058,kết thúc thời gian thử kết thúc thời gian thử ...,Luật lao động
1210,thẩm quyền giải quyết tranh chấp lao động cá n...,Luật lao động
1140,kỷ luật lao động kỷ luật lao động quy định tuâ...,Luật lao động


In [19]:
le = LabelEncoder()
df_train['label'] = le.fit_transform(df_train['label'])
df_train

,cleaned_text,label
2,áp dụng luật doanh nghiệp luật trường hợp luật...,1
3,bảo đảm nhà nước đối với doanh nghiệp chủ sở h...,1
5,quyền doanh nghiệp tự do kinh doanh ngành nghề...,1
8,đại diện pháp luật doanh nghiệp đại diện pháp ...,1
11,trách nhiệm đại diện ủy quyền chủ sở hữu thành...,1
...,...,...
1227,trường hợp đình công bất hợp pháp trường hợp đ...,7
1228,thông báo quyết định đóng cửa tạm thời làm việ...,7
1229,trường hợp cấm đóng cửa tạm thời làm việc thời...,7
1230,tiền lương quyền lợi hợp pháp lao động thời gi...,7


In [20]:
le = LabelEncoder()
df_test['label'] = le.fit_transform(df_test['label'])
df_test

,cleaned_text,label
62,tiền lương thù lao thưởng lợi ích chủ tịch hội...,1
40,chi nhánh văn phòng đại diện địa điểm kinh doa...,1
93,trách nhiệm chủ tịch thành viên hội đồng thành...,1
18,hồ sơ đăng ký công ty cổ phần giấy đề nghị đăn...,1
81,cơ cấu tổ chức quản lý công ty trách nhiệm hữu...,1
...,...,...
1180,gia hạn thu hồi giấy phép lao động giấy xác nh...,7
1125,khấu trừ tiền lương sử dụng lao động khấu trừ ...,7
1097,nguyên tắc thương lượng tập thể thương lượng t...,7
1092,học nghề tập nghề làm việc sử dụng lao động họ...,7


In [21]:
le = LabelEncoder()
df_val['label'] = le.fit_transform(df_val['label'])
df_val

,cleaned_text,label
32,định giá tài sản góp vốn tài sản góp vốn đồng ...,1
1,đối tượng áp dụng doanh nghiệp cơ quan tổ chức...,1
69,công bố thông tin công ty trách nhiệm hữu hạn ...,1
6,nghĩa vụ doanh nghiệp doanh nghiệp xã hội đáp ...,1
77,chủ tịch công ty chủ tịch công ty chủ sở hữu c...,1
...,...,...
1094,tổ chức đối thoại làm việc đối thoại làm việc ...,7
1058,kết thúc thời gian thử kết thúc thời gian thử ...,7
1210,thẩm quyền giải quyết tranh chấp lao động cá n...,7
1140,kỷ luật lao động kỷ luật lao động quy định tuâ...,7


In [22]:
df_train['label'].value_counts()

5    159
7    151
0    122
3     93
8     93
2     65
1     48
6     47
4     13
Name: label, dtype: int64

In [23]:
df_test['label'].value_counts()

1    27
4    27
3    27
0    27
6    27
2    27
5    27
8    27
7    27
Name: label, dtype: int64

In [24]:
df_val['label'].value_counts()

1    22
4    22
3    22
0    22
6    22
2    22
5    22
8    22
7    22
Name: label, dtype: int64

In [25]:
X_train = df_train['cleaned_text']
y_train = df_train['label']

X_test = df_test['cleaned_text']
y_test = df_test['label']

X_val = df_val['cleaned_text']
y_val = df_val['label']

print(f'Number samples of training set: {X_train.shape}')
print(f'Number samples of test set: {X_val.shape}')
print(f'Number samples of test set: {X_test.shape}')

Number samples of training set: (791,)
Number samples of test set: (198,)
Number samples of test set: (243,)


In [26]:

X_train = tfidf.fit_transform(X_train)
X_val = tfidf.transform(X_val)
X_test = tfidf.transform(X_test)

# TRANING MODEL

In [27]:
def evaluation(model, feature, label, title, metric1, metric2):
    input_prediction = model.predict(feature)
    input_accuracy = metric1(input_prediction, label)
    print(f'{title}: {input_accuracy}')
    print('='*55)
    print(metric2(label, input_prediction))

In [28]:
# Nối 2 dataframe dữ liệu df_train, df_val để dùng cho cross vali
train_df =  pd.concat([df_train, df_val], axis=0)
train_df

,cleaned_text,label
2,áp dụng luật doanh nghiệp luật trường hợp luật...,1
3,bảo đảm nhà nước đối với doanh nghiệp chủ sở h...,1
5,quyền doanh nghiệp tự do kinh doanh ngành nghề...,1
8,đại diện pháp luật doanh nghiệp đại diện pháp ...,1
11,trách nhiệm đại diện ủy quyền chủ sở hữu thành...,1
...,...,...
1094,tổ chức đối thoại làm việc đối thoại làm việc ...,7
1058,kết thúc thời gian thử kết thúc thời gian thử ...,7
1210,thẩm quyền giải quyết tranh chấp lao động cá n...,7
1140,kỷ luật lao động kỷ luật lao động quy định tuâ...,7


## 1. Support Vector Machine (SVM)

### 1.1. Defalt hyperparameters

In [29]:
svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [30]:
X_train_pred = svc.predict(X_train)
X_test_pred = svc.predict(X_test)
print(classification_report(X_train_pred, y_train))
print(classification_report(X_test_pred, y_test))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      0.97      0.98        96
           4       0.92      1.00      0.96        12
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       0.97      0.99      0.98        91

    accuracy                           0.99       791
   macro avg       0.99      1.00      0.99       791
weighted avg       1.00      0.99      0.99       791

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        30
           1       0.96      0.93      0.95        28
           2       0.96      0.96      0.96        27
           3       1.00      1.00      1.00        27
           4       0.56 

In [31]:
print(f'Accuracy: {round(accuracy_score(X_test_pred, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_pred, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_pred, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_pred, y_test, average="weighted"), 4)}')

Accuracy: 0.93
Precision: 0.9523
Recall: 0.93
F1-score: 0.9349


### 1.2 Best hyperparameters (Cao hơn)

In [32]:
# # Định nghĩa các siêu tham số cần tinh chỉnh
# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'gamma': [1, 0.1, 0.01, 0.001],
#     'kernel': ['rbf', 'linear']
# }

# # Định nghĩa GridSearchCV
# grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# # Huấn luyện mô hình
# grid_search.fit(X_val, y_val)

# # In ra các siêu tham số tốt nhất
# print("Best parameters found: ", grid_search.best_params_)

In [33]:
svc_hp = SVC( C= 1, gamma = 1, kernel= 'linear')
svc_hp.fit(X_train, y_train)

SVC(C=1, gamma=1, kernel='linear')

In [34]:
X_train_pred_hp = svc_hp.predict(X_train)
X_test_pred_hp = svc_hp.predict(X_test)
print(classification_report(X_train_pred_hp, y_train))
print(classification_report(X_test_pred_hp, y_test))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      0.97      0.98        96
           4       0.92      1.00      0.96        12
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       0.97      0.99      0.98        91

    accuracy                           0.99       791
   macro avg       0.99      1.00      0.99       791
weighted avg       1.00      0.99      0.99       791

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        30
           1       0.96      0.93      0.95        28
           2       0.96      0.96      0.96        27
           3       1.00      1.00      1.00        27
           4       0.78 

In [35]:
print(f'Accuracy: {round(accuracy_score(X_test_pred_hp, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_pred_hp, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_pred_hp, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_pred_hp, y_test, average="weighted"), 4)}')

Accuracy: 0.9547
Precision: 0.9607
Recall: 0.9547
F1-score: 0.9556


## 2. Random Forest

### 2.1 Default hyperparameters

In [36]:
rfc_default = RandomForestClassifier()
rfc_default.fit(X_train, y_train)

RandomForestClassifier()

In [37]:

X_train_pred_rf = rfc_default .predict(X_train)
X_test_pred_rf = rfc_default.predict(X_test)
print(classification_report(X_train_pred_rf, y_train))
print(classification_report(X_test_pred_rf, y_test))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      1.00      1.00        93
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       1.00      1.00      1.00        93

    accuracy                           1.00       791
   macro avg       1.00      1.00      1.00       791
weighted avg       1.00      1.00      1.00       791

              precision    recall  f1-score   support

           0       1.00      0.93      0.96        29
           1       0.96      0.87      0.91        30
           2       0.96      0.96      0.96        27
           3       1.00      1.00      1.00        27
           4       0.37 

In [38]:
print(f'Accuracy: {round(accuracy_score(X_test_pred_rf, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_pred_rf, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_pred_rf, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_pred_rf, y_test, average="weighted"), 4)}')

Accuracy: 0.9136
Precision: 0.9573
Recall: 0.9136
F1-score: 0.925


### 2.2 Best hyperparameters (Chấp nhận)

In [39]:
# # Định nghĩa mô hình và grid search
# rf = RandomForestClassifier(random_state=42)
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 20, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'bootstrap': [True, False],
#     'criterion': ['gini', 'entropy']
# }

# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
#                            cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# # Huấn luyện mô hình
# grid_search.fit(X_val, y_val)
# # Get the best hyperparameters from the grid search
# best_params = grid_search.best_params_



In [40]:
# print("Best Hyperparameters:", best_params)

In [41]:
# # Best Hyperparameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
rfc_best_hp = RandomForestClassifier(bootstrap = False, criterion = 'entropy', max_depth = None, max_features = 'sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200)
rfc_best_hp.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, criterion='entropy', n_estimators=200)

In [42]:
X_train_rfc_best_hp  = rfc_best_hp .predict(X_train)
X_test_rfc_best_hp  = rfc_best_hp .predict(X_test)
print(classification_report(X_train_rfc_best_hp, y_train))
print(classification_report(X_test_rfc_best_hp,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      1.00      1.00        93
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       1.00      1.00      1.00        93

    accuracy                           1.00       791
   macro avg       1.00      1.00      1.00       791
weighted avg       1.00      1.00      1.00       791

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        28
           1       0.96      0.90      0.93        29
           2       1.00      0.96      0.98        28
           3       1.00      1.00      1.00        27
           4       0.56 

In [43]:
print(f'Accuracy: {round(accuracy_score(X_test_rfc_best_hp, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_rfc_best_hp, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_rfc_best_hp, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_rfc_best_hp, y_test, average="weighted"), 4)}')

Accuracy: 0.9383
Precision: 0.9599
Recall: 0.9383
F1-score: 0.9432


## 3.Logistic Regression

### 3.1 Defalt hyperparameters

In [44]:
log_reg_default = LogisticRegression()
log_reg_default.fit(X_train, y_train)

LogisticRegression()

In [45]:
X_train_pred_lg = log_reg_default .predict(X_train)
X_test_pred_lg = log_reg_default.predict(X_test)
print(classification_report(X_train_pred_rf, y_train))
print(classification_report(X_test_pred_rf, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      1.00      1.00        93
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       1.00      1.00      1.00        93

    accuracy                           1.00       791
   macro avg       1.00      1.00      1.00       791
weighted avg       1.00      1.00      1.00       791

              precision    recall  f1-score   support

           0       1.00      0.93      0.96        29
           1       0.96      0.87      0.91        30
           2       0.96      0.96      0.96        27
           3       1.00      1.00      1.00        27
           4       0.37 

In [46]:
print(f'Accuracy: {round(accuracy_score(X_test_pred_lg, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_pred_lg, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_pred_lg, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_pred_lg, y_test, average="weighted"), 4)}')

Accuracy: 0.9177
Precision: 0.9569
Recall: 0.9177
F1-score: 0.9275


## 3.2 Tuning hyperparameter (bỏ - Tuning còn thấp hơn trước khi tuning)

In [47]:
# param_grid = {
#     'penalty': ['l1', 'l2'],
#     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
#     'solver': ['liblinear', 'newton-cg', 'lbfgs'],
#     'max_iter': [100, 200, 300, 400, 500],
#     'fit_intercept': [True, False]
# }

# log_reg = LogisticRegression()

# grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_val, y_val)
# best_params = grid_search.best_params_
# print("Best hyperparameters:", best_params)

In [48]:
# Best hyperparameters: {'C': 0.1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
log_reg_best_hps = LogisticRegression(C=0.1, fit_intercept=True, max_iter=100, penalty='l2', solver='liblinear')
log_reg_best_hps.fit(X_train, y_train)


LogisticRegression(C=0.1, solver='liblinear')

In [49]:
X_train_bs_hp = log_reg_best_hps.predict(X_train)
X_test_bs_hp = log_reg_best_hps.predict(X_test)
print(classification_report(X_train_bs_hp, y_train))
print(classification_report(X_test_bs_hp,y_test))

              precision    recall  f1-score   support

           0       0.98      0.92      0.94       130
           1       0.60      1.00      0.75        29
           2       0.83      0.96      0.89        56
           3       0.95      0.95      0.95        93
           4       0.00      0.00      0.00         0
           5       1.00      0.88      0.94       181
           6       0.83      1.00      0.91        39
           7       1.00      0.86      0.92       176
           8       0.91      0.98      0.94        87

    accuracy                           0.92       791
   macro avg       0.79      0.84      0.81       791
weighted avg       0.95      0.92      0.93       791

              precision    recall  f1-score   support

           0       0.96      0.90      0.93        29
           1       0.59      1.00      0.74        16
           2       0.93      1.00      0.96        25
           3       1.00      0.93      0.96        29
           4       0.00 

c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anac

In [50]:
print(f'Accuracy: {round(accuracy_score(X_test_bs_hp, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_bs_hp, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_bs_hp, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_bs_hp, y_test, average="weighted"), 4)}')

Accuracy: 0.8066
Precision: 0.9389
Recall: 0.8066
F1-score: 0.8436


c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 4. Navie Bayes

### 4.1 Default

In [51]:
navie = MultinomialNB()
navie.fit(X_train,y_train)

MultinomialNB()

In [52]:
X_train_df_nb = navie.predict(X_train)
X_test_df_nb = navie.predict(X_test)
print(classification_report(X_train_df_nb, y_train))
print(classification_report(X_test_df_nb,y_test))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       128
           1       0.67      1.00      0.80        32
           2       0.86      1.00      0.93        56
           3       0.98      0.95      0.96        96
           4       0.00      0.00      0.00         0
           5       1.00      0.87      0.93       183
           6       0.89      1.00      0.94        42
           7       0.99      0.90      0.94       167
           8       0.92      0.99      0.96        87

    accuracy                           0.93       791
   macro avg       0.81      0.85      0.83       791
weighted avg       0.96      0.93      0.94       791

              precision    recall  f1-score   support

           0       0.96      0.93      0.95        28
           1       0.70      1.00      0.83        19
           2       0.96      1.00      0.98        26
           3       1.00      0.93      0.96        29
           4       0.00 

c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anac

In [53]:
print(f'Accuracy: {round(accuracy_score(X_test_df_nb, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_df_nb, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_df_nb, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_df_nb, y_test, average="weighted"), 4)}')

Accuracy: 0.8107
Precision: 0.938
Recall: 0.8107
F1-score: 0.8459


c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 4.2 Hyperparameter (Chấp nhận - cao hơn nhiều)

In [54]:
# # Định nghĩa bộ tham số cho GridSearchCV
# param_grid = {
#     'alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
#     'fit_prior': [True, False]
# }

# # Tạo đối tượng MultinomialNB
# nb_model = MultinomialNB()

# # Tạo đối tượng GridSearchCV
# grid_search = GridSearchCV(estimator=nb_model, param_grid=param_grid, cv=5, scoring='accuracy')

# # Fit GridSearchCV vào dữ liệu huấn luyện
# grid_search.fit(X_val, y_val)

# best_params = grid_search.best_params_
# print("Best hyperparameters:", best_params)


In [55]:
# Best hyperparameters: {'alpha': 1.0, 'fit_prior': True}
navie_best_hps = MultinomialNB(alpha = 1.0, fit_prior = True)
navie_best_hps.fit(X_train, y_train)

MultinomialNB()

In [56]:
X_train_hps_nb = navie_best_hps.predict(X_train)
X_test_hps_nb = navie_best_hps.predict(X_test)
print(classification_report(X_train_hps_nb, y_train))
print(classification_report(X_test_hps_nb,y_test))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       128
           1       0.67      1.00      0.80        32
           2       0.86      1.00      0.93        56
           3       0.98      0.95      0.96        96
           4       0.00      0.00      0.00         0
           5       1.00      0.87      0.93       183
           6       0.89      1.00      0.94        42
           7       0.99      0.90      0.94       167
           8       0.92      0.99      0.96        87

    accuracy                           0.93       791
   macro avg       0.81      0.85      0.83       791
weighted avg       0.96      0.93      0.94       791

              precision    recall  f1-score   support

           0       0.96      0.93      0.95        28
           1       0.70      1.00      0.83        19
           2       0.96      1.00      0.98        26
           3       1.00      0.93      0.96        29
           4       0.00 

c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anac

In [57]:
print(f'Accuracy: {round(accuracy_score(X_test_hps_nb, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_hps_nb, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_hps_nb, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_hps_nb, y_test, average="weighted"), 4)}')

Accuracy: 0.8107
Precision: 0.938
Recall: 0.8107
F1-score: 0.8459


c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 5. Neural Network

### 5.1 Default

In [58]:
neural = MLPClassifier()
neural.fit(X_train, y_train)

MLPClassifier()

In [59]:
X_train_pred_nn = neural.predict(X_train)
X_test_pred_nn = neural.predict(X_test)
print(classification_report(X_train_pred_nn, y_train))
print(classification_report(X_test_pred_nn,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      1.00      1.00        93
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       1.00      1.00      1.00        93

    accuracy                           1.00       791
   macro avg       1.00      1.00      1.00       791
weighted avg       1.00      1.00      1.00       791

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        30
           1       1.00      0.93      0.96        29
           2       0.96      0.96      0.96        27
           3       1.00      0.96      0.98        28
           4       0.67 

In [60]:
print(f'Accuracy: {round(accuracy_score(X_test_pred_nn, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_pred_nn, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_pred_nn, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_pred_nn, y_test, average="weighted"), 4)}')

Accuracy: 0.9424
Precision: 0.9553
Recall: 0.9424
F1-score: 0.9452


### 5.2 Tuning hyper parameter (Cao hơn)

In [61]:

# # Định nghĩa mô hình MLPClassifier
# mlp = MLPClassifier(max_iter=100)

# # Định nghĩa các hyperparameters để tuning
# param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (50, 50)],
#     'activation': ['tanh', 'relu'],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001, 0.05],
#     'learning_rate': ['constant', 'adaptive'],
# }

# # Sử dụng GridSearchCV để tìm các hyperparameters tốt nhất
# grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, n_jobs=-1, cv=3)
# grid_search.fit(X_val, y_val)

# best_params = grid_search.best_params_
# print("Best hyperparameters:", best_params)


In [62]:
# Best hyperparameters: {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'solver': 'adam'}

nn_best_hps = MLPClassifier(activation = 'tanh', alpha = 0.05, hidden_layer_sizes = (100,), learning_rate = 'adaptive', solver = 'adam')
nn_best_hps.fit(X_train, y_train)

c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='tanh', alpha=0.05, learning_rate='adaptive')

In [63]:
X_train_best_nn = nn_best_hps.predict(X_train)
X_test_best_nn = nn_best_hps.predict(X_test)
print(classification_report(X_train_pred_nn, y_train))
print(classification_report(X_test_pred_nn,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      1.00      1.00        93
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       1.00      1.00      1.00        93

    accuracy                           1.00       791
   macro avg       1.00      1.00      1.00       791
weighted avg       1.00      1.00      1.00       791

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        30
           1       1.00      0.93      0.96        29
           2       0.96      0.96      0.96        27
           3       1.00      0.96      0.98        28
           4       0.67 

In [64]:
print(f'Accuracy: {round(accuracy_score(X_test_best_nn, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_best_nn, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_best_nn, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_best_nn, y_test, average="weighted"), 4)}')

Accuracy: 0.9547
Precision: 0.9627
Recall: 0.9547
F1-score: 0.9563


# II. Cross Validation

In [65]:
# Nối 2 file dữ liệu có sẵn df_train và df_val
df_train = pd.concat([df_train, df_val], ignore_index=True)

In [75]:
# Số lượng folds cho K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Danh sách các mô hình
models = {
    'SVM': SVC(C= 1, gamma = 1, kernel= 'linear'),
    'Naive Bayes': MultinomialNB(alpha = 1.0, fit_prior = True),
    'Random Forest': RandomForestClassifier(bootstrap = False, criterion = 'entropy', max_depth = None, max_features = 'sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200),
    'Logistic Regression': LogisticRegression(max_iter=200),
    'Neural Network': MLPClassifier(activation = 'tanh', alpha = 0.05, hidden_layer_sizes = (100,), learning_rate = 'adaptive', solver = 'adam')
}

# Lưu trữ kết quả cross-validation
results = {}
for model_name, model in models.items():
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results[model_name] = cv_results
    print(f"{model_name}: Mean Accuracy = {np.mean(cv_results)}, Std = {np.std(cv_results)}")

# Huấn luyện mô hình tốt nhất và đánh giá trên tập test
best_model_name = max(results, key=lambda k: np.mean(results[k]))
best_model = models[best_model_name]
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Đánh giá mô hình tốt nhất
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred, average='weighted')
test_recall = recall_score(y_test, y_pred, average='weighted')
test_f1 = f1_score(y_test, y_pred, average='weighted')

print(f"\nBest Model: {best_model_name}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1}")

# Đánh giá tất cả các mô hình trên tập test
print("\nAll models evaluation on test set:")
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred, average='weighted')
    test_recall = recall_score(y_test, y_pred, average='weighted')
    test_f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"{model_name} Test Accuracy: {test_accuracy}")
    print(f"{model_name} Test Precision: {test_precision}")
    print(f"{model_name} Test Recall: {test_recall}")
    print(f"{model_name} Test F1 Score: {test_f1}")

SVM: Mean Accuracy = 0.951970384523525, Std = 0.008512163000845835
Naive Bayes: Mean Accuracy = 0.8938141867685694, Std = 0.017571369939366633
Random Forest: Mean Accuracy = 0.9582994984475757, Std = 0.011646555089199536
Logistic Regression: Mean Accuracy = 0.9494467001034949, Std = 0.014363489234271658


c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

Neural Network: Mean Accuracy = 0.959533476634026, Std = 0.011769035051660123


c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(



Best Model: Neural Network
Test Accuracy: 0.9465020576131687
Test Precision: 0.9510195574611284
Test Recall: 0.9465020576131687
Test F1 Score: 0.9447617339249208

All models evaluation on test set:
SVM Test Accuracy: 0.9547325102880658
SVM Test Precision: 0.9576425631981188
SVM Test Recall: 0.9547325102880658
SVM Test F1 Score: 0.953830439649153
Naive Bayes Test Accuracy: 0.8106995884773662
Naive Bayes Test Precision: 0.7761683335821267
Naive Bayes Test Recall: 0.8106995884773662
Naive Bayes Test F1 Score: 0.7755047721589117


c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest Test Accuracy: 0.934156378600823
Random Forest Test Precision: 0.941751325411456
Random Forest Test Recall: 0.934156378600823
Random Forest Test F1 Score: 0.9303758381271409
Logistic Regression Test Accuracy: 0.9176954732510288
Logistic Regression Test Precision: 0.9322659824575533
Logistic Regression Test Recall: 0.9176954732510288
Logistic Regression Test F1 Score: 0.9079037366683564
Neural Network Test Accuracy: 0.9547325102880658
Neural Network Test Precision: 0.9575432301485942
Neural Network Test Recall: 0.9547325102880658
Neural Network Test F1 Score: 0.9531975367983829


c:\Users\ADMIN\anaconda3\envs\AdvancedPython\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Save the best model

In [71]:
import pickle

# Lưu mô hình vào tệp tin 'rfc_pretrain.pkl'
with open('./best_nn_model.pkl', 'wb') as file:
    pickle.dump(nn_best_hps, file)


In [72]:
with open('./class_encoder.pkl', 'wb') as file:
    pickle.dump(le, file)

In [73]:
with open('./vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf, file)

In [74]:
le.inverse_transform([1])[0]

'LUẬT DOANH NGHIỆP'